### 图像透视变换
- 输入图像，顺时针依次用鼠标点击四个点，作为仿射变换后和原图左上、右上、右下、左下四个点对应的点。
- 按下Enter键，对图像做透视变换。
- 将边界黑色背景区域设置为透明，保存新的图像。

In [23]:
import cv2
import numpy as np

# 创建一个全局变量来保存点击的点
points = []

# 鼠标回调函数，用于获取鼠标点击的坐标
def select_points(event, x, y, flags, param):
    global points, img_copy
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        # 更新显示的图像
        img_copy = img.copy()
        for point in points:
            cv2.circle(img_copy, point, 5, (0, 255, 0), -1)
        cv2.imshow("image", img_copy)

# 读取输入图像
img = cv2.imread('./input/test.png')
img_copy = img.copy()

# 显示图像并获取用户点击的四个点
cv2.imshow("image", img)
cv2.setMouseCallback("image", select_points)

while True:
    if len(points) == 4:
        # 按下 Enter 键继续
        if cv2.waitKey(1) & 0xFF == 13:
            print("Performing perspective transformation...")
            break
    cv2.waitKey(1)

cv2.destroyAllWindows()

# 定义目标矩形的大小、输出图像的位置
width, height = 500, 500
pts1 = np.float32(points)
pts2 = np.float32([[0, 0], [width, 0], [width, height], [0, height]])

# 计算透视变换矩阵
M = cv2.getPerspectiveTransform(pts1, pts2)
# 进行透视变换
warped_img = cv2.warpPerspective(img, M, (width, height))

# 将黑色背景区域设置为透明
gray = cv2.cvtColor(warped_img, cv2.COLOR_BGR2GRAY)
_, alpha = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)  # 使用阈值1来处理几乎黑色的区域
b, g, r = cv2.split(warped_img)
rgba = [b, g, r, alpha]
warped_img_with_alpha = cv2.merge(rgba, 4)

# 保存结果
cv2.imwrite('./output/2_3/warped_img_with_alpha.png', warped_img_with_alpha)

# 显示结果
cv2.imshow('Warped Image', warped_img_with_alpha)
cv2.waitKey(0)
cv2.destroyAllWindows()


Performing perspective transformation...


### 在2D平面可视化人体骨架数据
- 基于某一动作的3D坐标序列画出该动作在一个2D平面上的投影
- 要求模拟相机成像过程，设定相机的内参和外参
- 不同关节点采用不同颜色表示，关节点之间的连线用两个节点的中间颜色

设置模拟相机参数矩阵

In [25]:
import numpy as np
import cv2

# 模拟相机的内参矩阵 (Intrinsic Parameters)
focal_length = 800  # 焦距
principal_point = (400, 300)  # 主点（成像平面中心）
camera_matrix = np.array([[focal_length, 0, principal_point[0]],
                          [0, focal_length, principal_point[1]],
                          [0, 0, 1]], dtype=np.float64)

# 模拟相机的外参矩阵 (Extrinsic Parameters)
rotation_vector = np.array([0.0, 0.0, 0.0], dtype=np.float64)  # 无旋转
translation_vector = np.array([0.0, 0.0, -1000.0], dtype=np.float64)  # 沿Z轴向后移1000单位

加载3D骨架数据，并建立骨架连接关系

In [26]:

# 加载3D骨架数据
data = np.load('./input/brushing_hair.npy')

# 骨架连接关系
connections = [
    (1, 2), (2, 21), (21, 3), (3, 4),  # 脊柱
    (21, 5), (5, 6), (6, 7), (7, 8), (8, 23), (23, 22),  # 右臂
    (21, 9), (9, 10), (10, 11), (11, 12), (12, 25), (25, 24),  # 左臂
    (13, 14), (14, 15), (15, 16),  # 右腿
    (17, 18), (18, 19), (19, 20),  # 左腿
]

# 选择要可视化的帧
frame_index = 0  # 可视化第一个动作帧
skeleton_3d = data[frame_index]


添加合适的缩放因子，使2D投影结果更加便于观察

In [27]:

# 选择一个合适的缩放因子
scale_factor = 150

# 对 3D 坐标进行缩放
skeleton_3d_scaled = skeleton_3d * scale_factor

# 将缩放后的 3D 点投影到 2D 平面
# 将3D点投影到2D平面
skeleton_2d, _ = cv2.projectPoints(skeleton_3d_scaled, rotation_vector, translation_vector, camera_matrix, distCoeffs=None)

# 保留小数形式的坐标
skeleton_2d = skeleton_2d.reshape(-1, 2)


将关节设置为不同的颜色

In [28]:

# 创建空白图像（背景）
image = np.ones((600, 800, 3), dtype=np.uint8) * 255  # 白色背景

joint_colors = [tuple(map(int, np.random.randint(0, 255, 3))) for _ in range(len(skeleton_2d))]

for i, point in enumerate(skeleton_2d):
    cv2.circle(image, tuple(np.round(point).astype(int)), 8, joint_colors[i], -1)


for connection in connections:
    pt1 = np.round(skeleton_2d[connection[0] - 1]).astype(int)
    pt2 = np.round(skeleton_2d[connection[1] - 1]).astype(int)

    color1 = np.array(joint_colors[connection[0] - 1])
    color2 = np.array(joint_colors[connection[1] - 1])
    mid_color = tuple(map(int, ((color1 + color2) // 2)))

    cv2.line(image, tuple(pt1), tuple(pt2), mid_color, 4)


# 显示图像
cv2.imshow('Skeleton Projection', image)
cv2.imwrite('./output/2_3/skeleton_projection.png', image)
cv2.waitKey(0)
cv2.destroyAllWindows()